In [1]:
%pip install wandb
%pip install ultralytics
%pip install matplotlib
%pip install numpy
%pip install opencv-python
%pip install inference

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
ZERO: int = 0
ONE: int = 1

ITERATION: int  = 1
IOU: int        = 80
EPOCH: int      = 6
BATCHES: int    = 9

In [3]:
def calculateIOU() -> float:
    global IOU

    return float(
        IOU
        /
        100
    )

In [4]:
from ultralytics    \
    import YOLO

from math           \
    import (
        floor
)

from random         \
    import (
        SystemRandom
)

from os.path        \
    import (
        join
)

from os \
    import environ

from logging import ( 
    getLogger, 
    ERROR
)

from wandb import (
    init, 
    finish
)

from wandb.integration.ultralytics \
    import (
        add_wandb_callback
)

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [5]:
environ['WANDB_DIR'] = '~/wandb'

environ['WANDB_CONSOLE']       = 'off'
environ['WANDB_DISABLE_CODE']  = 'off'
environ['WANDB_NOTEBOOK_NAME'] = 'TrainingOfModel.ipynb'
environ['WANDB_ENTITY']        = 'Designermadsen'
environ['WANDB_SILENT']        = 'True'


environ['YOLO_MODEL_ITERATION'] = str(
    ITERATION
)

environ['YOLO_WEIGHT_MODEL_VERSION'] = \
    str(
        '/mnt/c/Users/Kentv/Desktop/Spectre/runs/detect/train6/weights/best.pt'
)

environ['YOLO_MODEL_VERSION']        = \
    'yolo11m.pt'



environ['TRAINING_EPOCH'] = str(
    EPOCH
)

environ['TRAINING_BATCHES'] = str(
    BATCHES
)

environ['TRAINING_PATIENCE'] = str(
    0
)

environ['TRAINING_WORKERS'] = str(
    20
)

In [6]:
def get_weight_location() -> str:
    global MODEL_WEIGHT_LOCATION
    return MODEL_WEIGHT_LOCATION

def set_weight_location(
    value: str
) -> None:
    global MODEL_WEIGHT_LOCATION
    MODEL_WEIGHT_LOCATION = value

def get_current_model_iteration() -> int:
    global MODEL_ITERATION_VERSION
    return MODEL_ITERATION_VERSION

def set_current_model_iteration(
    value: int
) -> None:
    global MODEL_ITERATION_VERSION
    MODEL_ITERATION_VERSION = value

def increment_of_model() -> None:
    global ONE

    set_current_model_iteration(
        get_current_model_iteration() + ONE
    )

def generate_weight_location() -> str:
    weight_path = join(
        str(
            './runs/detect/train' 
        )        
        + 
        str(
            get_current_model_iteration()
        ),
        'weights/best.pt'
    )

    set_weight_location(
        weight_path
    )

    return get_weight_location()

def getSeed() -> int:
    global SEED
    
    if isSeedNone():
        return generateSeed()

    return SEED

def setSeed(
    value: int
) -> None:
    global SEED
    SEED = value


def generateGenerator() -> None:
    global generator

    if generator is None:
        generator = SystemRandom()

def isSeedNone() -> bool:
    global SEED
    return SEED is None

def maximumSizeForInteger() -> int:
    return 2147483647

def generateSeed() -> None:
    global          \
        SEED,       \
        generator,  \
        ZERO
    
    generateGenerator()

    setSeed(
        generator.randint(
            ZERO, 
            maximumSizeForInteger()
        )
    )

    return SEED

def getModel():
    global model
    return model

def setModel(
    value
):
    global model
    model = value

def modelLoadWeights() -> None:
    global MODEL_WEIGHT_LOCATION

    if not (
        MODEL_WEIGHT_LOCATION is None
    ):
        getModel().load(
            get_weight_location()
        )

def initializeLogging() -> None:
    logger = getLogger(
        __name__
    )

    logger.setLevel(
        ERROR
    )

In [7]:
generator   = None
model       = None
logger      = None
model       = None

NUMBER_OF_TIMES_TO_ITERATE_TRAINING: int = ONE

MODEL_ITERATION_VERSION: int = int(
    environ['YOLO_MODEL_ITERATION']
)

FIRST_MODEL_TO_LOAD: str = environ['YOLO_WEIGHT_MODEL_VERSION']

FINISH_WHEN_DONE: bool = True

EPOCHS: int     = int(
    environ['TRAINING_EPOCH']
)

BATCH: int      = int(
    environ['TRAINING_BATCHES']
)

PATIENCE: int   = int(
    environ['TRAINING_PATIENCE']
)


IMAGE_WIDTH_SIZE: int = 640
IMAGE_HEIGHT_SIZE: int = 640

IMAGE_DIMENSIONS: tuple = (
    IMAGE_HEIGHT_SIZE, 
    IMAGE_WIDTH_SIZE
)

SEED = None

IS_TO_RESUME: bool = False

#
MOSAIC_END_AT: int = -ONE

PROJECT_NAME: str = 'spectre'

HOME_LOCATION: str = '/home/ai'

DATASET_LOCATION: str = join(
    HOME_LOCATION,
    'dataset.yaml'
)

MODEL_WEIGHT_LOCATION: str | None = None

#
DEVICES_TO_USE: list = [
    ZERO
]

In [8]:
def getTrainingWorkers() -> int:
    return int(
        environ['TRAINING_WORKERS']
    )

In [9]:
initializeLogging()

init(
    project     = PROJECT_NAME, 
    job_type    = 'training',
    save_code   = False
)

setModel(
    YOLO(
        FIRST_MODEL_TO_LOAD,
        verbose = False
    )
)

add_wandb_callback(
    model,
    enable_train_validation_logging     = False,
    enable_validation_logging           = False,
    enable_prediction_logging           = False,
    enable_model_checkpointing          = False
)


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find TrainingOfModel.ipynb.


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

In [ ]:
if not IS_TO_RESUME:
    generateSeed()

for idx in range(
    NUMBER_OF_TIMES_TO_ITERATE_TRAINING
):
    training_results =                          \
        model.train(
            data         = DATASET_LOCATION, 

            batch        = int(
                environ['TRAINING_BATCHES']
            ), 
            epochs       = int(
                environ['TRAINING_EPOCH']
            ),
            cos_lr       = True,
            box          = 3.0,
            iou          = calculateIOU(),
            dropout      = 0.075,
            patience     = PATIENCE,
            
            lrf          = 0.01,
            conf         = 0.001,
            weight_decay = 0.001,

            auto_augment = 'randaugment',
            bgr          = 0.05,
            hsv_h        = 0.15,
            hsv_s        = 0.15,
            hsv_v        = 0.15,

            translate    = 0.05,
            scale        = 0.05,
            mixup        = 0.40,
            momentum     = 0.95,
            mosaic       = 0.40,
            
            close_mosaic  = MOSAIC_END_AT,
            
            imgsz         = IMAGE_DIMENSIONS,
            multi_scale   = False,
            resume        = IS_TO_RESUME,
            seed          = getSeed(),
            workers       = getTrainingWorkers(),
            stream_buffer = True,
            device        = DEVICES_TO_USE,
            
            save          = True,

            plots         = True,
            show          = False,
            verbose       = False
    )
    
    if NUMBER_OF_TIMES_TO_ITERATE_TRAINING > ONE:
        increment_of_model()

    if not (
        int( 
            NUMBER_OF_TIMES_TO_ITERATE_TRAINING 
            - 
            ONE
        )
        == 
        idx
    ):
        modelLoadWeights()


New https://pypi.org/project/ultralytics/8.3.76 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.72 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=/mnt/c/Users/Kentv/Desktop/Spectre/runs/detect/train6/weights/best.pt, data=/home/ai/dataset.yaml, epochs=6, time=None, patience=0, batch=9, imgsz=(640, 640), save=True, save_period=-1, cache=False, device=[0], workers=20, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=1134057815, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=-1, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.075, val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.8, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=True, visualize=False, augment=False, agn

train: Scanning /opt/dataset/tmod/training/labels.cache... 6400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6400/6400 [00:00<?, ?it/s]
val: Scanning /opt/dataset/tmod/validation/labels.cache... 1600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1600/1600 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.95' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.000984375), 112 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 20 dataloader workers
Logging results to runs/detect/train7
Starting training for 6 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/6      5.24G     0.2095     0.3598      1.034          1        640: 100%|██████████| 712/712 [03:45<00:00,  3.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 89/89 [00:19<00:00,  4.65it/s]

                   all       1600       1868      0.971      0.926      0.969      0.764


...

        4/6      5.21G     0.2071       0.35      1.025          1        640: 100%|██████████| 712/712 [03:38<00:00,  3.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 89/89 [00:17<00:00,  5.22it/s]

                   all       1600       1868      0.883      0.926      0.936      0.745



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/6      5.22G     0.2139     0.3598      1.044          5        640: 100%|██████████| 712/712 [03:35<00:00,  3.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 89/89 [00:24<00:00,  3.61it/s]

                   all       1600       1868      0.978      0.944      0.967       0.77


In [ ]:
if FINISH_WHEN_DONE:
    finish()